In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import logging
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (r2_score, mean_absolute_error, mean_absolute_percentage_error,
                            mean_squared_error, explained_variance_score)
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import joblib
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# ==============================================
# SETUP LOGGING AND OUTPUT FOLDER
# ==============================================

# Create output folder with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_folder = f'Results_Model_Comparison_{timestamp}'
os.makedirs(output_folder, exist_ok=True)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(output_folder, 'execution.log')),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

logger.info(f"Script execution started. Output will be saved in: {output_folder}")

# ==============================================
# DATA LOADING AND PREPROCESSING
# ==============================================

def load_and_preprocess_data(data_path, target='SeaTemperature', selected_features=None):
    """Load and preprocess the dataset, selecting specified features and scaling data."""
    try:
        logger.info("Loading and preprocessing dataset...")

        # Load the CSV file
        df = pd.read_csv(data_path)
        logger.info(f"Raw dataset shape: {df.shape}")
        logger.info("\nFirst 5 rows before preprocessing:\n" + str(df.head()))

        # If selected_features is None, compute mutual information scores
        if selected_features is None:
            logger.info("No selected features provided. Computing mutual information scores...")
            features = [col for col in df.columns if col != target and df[col].dtype != 'object']
            X = df[features].fillna(df[features].mean())
            y = df[target]
            
            # Calculate mutual information scores
            mi_scores = mutual_info_regression(X, y)
            mi_scores_df = pd.DataFrame({'Feature': features, 'MI_Score': mi_scores})
            mi_scores_df = mi_scores_df.sort_values(by='MI_Score', ascending=False)
            
            # Select top 10 features
            selected_features = mi_scores_df.head(10)['Feature'].tolist()
            logger.info("Top 10 features based on MI scores:\n" + str(mi_scores_df.head(10)))

        # Ensure selected_features is a list
        if not isinstance(selected_features, list):
            raise ValueError("selected_features must be a list of feature names")

        # Verify all selected features exist in the dataset
        missing_features = [f for f in selected_features if f not in df.columns]
        if missing_features:
            raise ValueError(f"Features not found in dataset: {missing_features}")

        # Handle missing values by filling with column mean
        df[selected_features] = df[selected_features].fillna(df[selected_features].mean())
        logger.info("Missing values filled with column means.")

        # Split features and target
        X = df[selected_features].values
        y = df[target].values

        # Scale features and target
        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()
        X_scaled = scaler_X.fit_transform(X)
        y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

        # Save preprocessed data sample
        preprocessed_sample = pd.DataFrame(X_scaled, columns=selected_features)
        preprocessed_sample['target_scaled'] = y_scaled
        preprocessed_sample.head(20).to_csv(os.path.join(output_folder, 'preprocessed_data_sample.csv'), index=False)
        logger.info(f"Preprocessed data sample saved to: {output_folder}/preprocessed_data_sample.csv")

        return X_scaled, y_scaled, scaler_X, scaler_y, selected_features

    except Exception as e:
        logger.error(f"Error during data loading/preprocessing: {str(e)}", exc_info=True)
        raise

# ==============================================
# SEQUENCE CREATION
# ==============================================

def create_sequences(X, y, seq_length=24):
    """Create sequences for LSTM input."""
    try:
        logger.info(f"Creating sequences with sequence length: {seq_length}")
        X_seq, y_seq = [], []
        for i in range(len(X) - seq_length):
            X_seq.append(X[i:i + seq_length])
            y_seq.append(y[i + seq_length])
        X_seq, y_seq = np.array(X_seq), np.array(y_seq)
        logger.info(f"Sequence shapes - X: {X_seq.shape}, y: {y_seq.shape}")
        return X_seq, y_seq

    except Exception as e:
        logger.error(f"Error during sequence creation: {str(e)}", exc_info=True)
        raise

# ==============================================
# MODEL BUILDING
# ==============================================

def build_lstm_model(input_shape):
    """Build an improved LSTM model with Bidirectional layers for regression."""
    try:
        logger.info("Building LSTM model...")
        model = Sequential([
            Bidirectional(LSTM(256, activation='tanh', input_shape=input_shape, return_sequences=True)),
            Dropout(0.2),
            Bidirectional(LSTM(128, activation='tanh', return_sequences=True)),
            Dropout(0.2),
            Bidirectional(LSTM(64, activation='tanh')),
            Dropout(0.2),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(1)
        ])
        
        model.compile(optimizer=RMSprop(learning_rate=0.001), loss='mse', metrics=['mae'])
        model.build(input_shape=(None, input_shape[0], input_shape[1]))
        logger.info("LSTM model compiled and built successfully.")
        return model

    except Exception as e:
        logger.error(f"Error during model building: {str(e)}", exc_info=True)
        raise

def build_rf_model():
    """Build RandomForestRegressor model."""
    try:
        logger.info("Building RandomForestRegressor model...")
        model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
        logger.info("RandomForestRegressor model built successfully.")
        return model
    except Exception as e:
        logger.error(f"Error during RF model building: {str(e)}", exc_info=True)
        raise

def build_xgb_model():
    """Build XGBRegressor model."""
    try:
        logger.info("Building XGBRegressor model...")
        model = XGBRegressor(n_estimators=100, max_depth=10, learning_rate=0.1, random_state=42, n_jobs=-1)
        logger.info("XGBRegressor model built successfully.")
        return model
    except Exception as e:
        logger.error(f"Error during XGB model building: {str(e)}", exc_info=True)
        raise

# ==============================================
# MODEL EVALUATION AND VISUALIZATION
# ==============================================

def evaluate_and_visualize(model, model_name, X_test, y_test, scaler_y, output_folder, is_lstm=False):
    """Evaluate the model and generate visualizations for regression."""
    try:
        logger.info(f"Evaluating {model_name} model...")

        # Predict on test set
        if is_lstm:
            y_pred = model.predict(X_test, verbose=0)
        else:
            X_test_flat = X_test.reshape(X_test.shape[0], -1) if is_lstm else X_test
            y_pred = model.predict(X_test_flat)

        # Reshape predictions to 2D if they're 1D
        if len(y_pred.shape) == 1:
            y_pred = y_pred.reshape(-1, 1)
        if len(y_test.shape) == 1:
            y_test = y_test.reshape(-1, 1)

        # Inverse transform predictions and actual values
        y_test_inv = scaler_y.inverse_transform(y_test)
        y_pred_inv = scaler_y.inverse_transform(y_pred)

        # Regression metrics
        r2 = r2_score(y_test_inv, y_pred_inv)
        mae = mean_absolute_error(y_test_inv, y_pred_inv)
        mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100
        rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
        explained_var = explained_variance_score(y_test_inv, y_pred_inv)

        # Log metrics
        logger.info(f"\n{model_name} Test Set Regression Metrics:")
        logger.info(f"R² Score: {r2:.4f}")
        logger.info(f"Mean Absolute Error: {mae:.4f}")
        logger.info(f"Mean Absolute Percentage Error: {mape:.4f}%")
        logger.info(f"Root Mean Squared Error: {rmse:.4f}")
        logger.info(f"Explained Variance Score: {explained_var:.4f}")

        # Save metrics to file
        with open(os.path.join(output_folder, f'{model_name}_metrics.txt'), 'w') as f:
            f.write(f"{model_name} Regression Metrics:\n")
            f.write(f"R² Score: {r2:.4f}\n")
            f.write(f"Mean Absolute Error: {mae:.4f}\n")
            f.write(f"Mean Absolute Percentage Error: {mape:.4f}%\n")
            f.write(f"Root Mean Squared Error: {rmse:.4f}\n")
            f.write(f"Explained Variance Score: {explained_var:.4f}\n")

        # Visualizations
        # 1. Predicted vs Actual Plot
        plt.figure(figsize=(12, 6))
        plt.plot(y_test_inv, label='Actual Sea Temperature', alpha=0.7)
        plt.plot(y_pred_inv, label='Predicted Sea Temperature', alpha=0.7)
        plt.title(f'{model_name}: Actual vs Predicted Sea Temperature')
        plt.xlabel('Sample Index')
        plt.ylabel('Sea Temperature (°C)')
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, f'{model_name}_actual_vs_predicted.png'))
        plt.close()

        # 2. Error Distribution Plot
        errors = y_test_inv - y_pred_inv
        plt.figure(figsize=(10, 6))
        sns.histplot(errors, kde=True, bins=30)
        plt.title(f'{model_name}: Prediction Error Distribution')
        plt.xlabel('Prediction Error (°C)')
        plt.ylabel('Frequency')
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, f'{model_name}_error_distribution.png'))
        plt.close()

        # 3. Scatter Plot of Predicted vs Actual
        plt.figure(figsize=(8, 8))
        plt.scatter(y_test_inv, y_pred_inv, alpha=0.5)
        plt.plot([y_test_inv.min(), y_test_inv.max()], [y_test_inv.min(), y_test_inv.max()], 'r--', lw=2)
        plt.title(f'{model_name}: Predicted vs Actual Sea Temperature')
        plt.xlabel('Actual Sea Temperature (°C)')
        plt.ylabel('Predicted Sea Temperature (°C)')
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, f'{model_name}_scatter_pred_vs_actual.png'))
        plt.close()

        logger.info(f"{model_name} visualizations saved successfully.")

        return {
            'r2': r2, 'mae': mae, 'mape': mape, 'rmse': rmse, 'explained_var': explained_var
        }

    except Exception as e:
        logger.error(f"Error during {model_name} evaluation/visualization: {str(e)}", exc_info=True)
        raise

# ==============================================
# MAIN EXECUTION
# ==============================================

def main(data_path, selected_features=None):
    """Main function to train and evaluate LSTM, RandomForest, and XGBoost models."""
    try:
        # Step 1: Load and preprocess data
        X_scaled, y_scaled, scaler_X, scaler_y, feature_cols = load_and_preprocess_data(
            data_path, target='SeaTemperature', selected_features=selected_features
        )

        # Step 2: Create sequences for LSTM
        seq_length = 24
        X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

        # Step 3: Train-test split (80-20)
        train_size = int(0.8 * len(X_seq))
        X_train_seq, X_test_seq = X_seq[:train_size], X_seq[train_size:]
        y_train_seq, y_test_seq = y_seq[:train_size], y_seq[train_size:]
        
        # For RF and XGB, use flattened data
        X_train_flat = X_scaled[:train_size]
        X_test_flat = X_scaled[train_size:len(X_scaled)-seq_length]
        y_train_flat = y_scaled[:train_size]
        y_test_flat = y_scaled[train_size:len(y_scaled)-seq_length]
        
        logger.info("Data split - LSTM Train shape: {}, Test shape: {}".format(X_train_seq.shape, X_test_seq.shape))
        logger.info("Data split - RF/XGB Train shape: {}, Test shape: {}".format(X_train_flat.shape, X_test_flat.shape))
        logger.info(f"Selected features: {feature_cols}")

        # Step 4: Build models
        lstm_model = build_lstm_model((seq_length, len(feature_cols)))
        rf_model = build_rf_model()
        xgb_model = build_xgb_model()

        # Step 5: Train LSTM model with callbacks
        lstm_callbacks = [
            EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6, verbose=1),
            ModelCheckpoint(os.path.join(output_folder, 'best_lstm_sea_temperature_model.h5'),
                        monitor='val_loss', save_best_only=True, verbose=1)
        ]

        logger.info("Starting LSTM model training...")
        lstm_history = lstm_model.fit(
            X_train_seq, y_train_seq,
            epochs=200,
            batch_size=32,
            validation_split=0.25,
            callbacks=lstm_callbacks,
            verbose=2
        )

        # Step 6: Train RF and XGB models
        logger.info("Starting RandomForestRegressor training...")
        rf_model.fit(X_train_flat, y_train_flat.ravel())

        logger.info("Starting XGBRegressor training...")
        xgb_model.fit(X_train_flat, y_train_flat.ravel())

        # Step 7: Evaluate and visualize
        lstm_metrics = evaluate_and_visualize(lstm_model, 'LSTM', X_test_seq, y_test_seq, scaler_y, output_folder, is_lstm=True)
        rf_metrics = evaluate_and_visualize(rf_model, 'RandomForest', X_test_flat, y_test_flat, scaler_y, output_folder)
        xgb_metrics = evaluate_and_visualize(xgb_model, 'XGBoost', X_test_flat, y_test_flat, scaler_y, output_folder)

        # Step 8: Plot training history for LSTM
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(lstm_history.history['loss'], label='Training Loss')
        plt.plot(lstm_history.history['val_loss'], label='Validation Loss')
        plt.title('LSTM: Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(lstm_history.history['mae'], label='Training MAE')
        plt.plot(lstm_history.history['val_mae'], label='Validation MAE')
        plt.title('LSTM: Training and Validation MAE')
        plt.xlabel('Epoch')
        plt.ylabel('MAE')
        plt.legend()

        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, 'lstm_training_history.png'))
        plt.close()

        logger.info("LSTM training history plot saved.")

        # Step 9: Save models
        lstm_model.save(os.path.join(output_folder, 'final_lstm_sea_temperature_model.h5'))
        joblib.dump(rf_model, os.path.join(output_folder, 'final_rf_sea_temperature_model.pkl'))
        joblib.dump(xgb_model, os.path.join(output_folder, 'final_xgb_sea_temperature_model.pkl'))
        logger.info(f"Models saved to: {output_folder}")

        # Step 10: Save comparison summary
        comparison_df = pd.DataFrame({
            'Metric': ['R²', 'MAE', 'MAPE (%)', 'RMSE', 'Explained Variance'],
            'LSTM': [lstm_metrics['r2'], lstm_metrics['mae'], lstm_metrics['mape'], lstm_metrics['rmse'], lstm_metrics['explained_var']],
            'RandomForest': [rf_metrics['r2'], rf_metrics['mae'], rf_metrics['mape'], rf_metrics['rmse'], rf_metrics['explained_var']],
            'XGBoost': [xgb_metrics['r2'], xgb_metrics['mae'], xgb_metrics['mape'], xgb_metrics['rmse'], xgb_metrics['explained_var']]
        })
        comparison_df.to_csv(os.path.join(output_folder, 'model_comparison.csv'), index=False)
        logger.info("Model comparison summary saved.")

        logger.info("Pipeline completed successfully.")

        return lstm_model, rf_model, xgb_model, lstm_history, scaler_y, feature_cols

    except Exception as e:
        logger.error(f"Error in main execution: {str(e)}", exc_info=True)
        raise

if __name__ == "__main__":
    # Define data path
    data_path = r'download_35.csv'

    # Define selected features from RFE
    selected_features = [
        'longitude', 'latitude', 'month_sin', 'HeatIndex', 'SeaTemperature_lag1', 
        'SeaTemperature_lag2', 'SeaTemperature_rolling_3h', 'SeaTemperature_rolling_6h', 
        'SeaTemperature_rolling_12h', 'SeaTemp_WindSpeed_interaction'
    ]

    # Run the main function
    lstm_model, rf_model, xgb_model, lstm_history, scaler_y, feature_cols = main(data_path, selected_features)

2025-05-09 15:58:05,783 - INFO - Script execution started. Output will be saved in: Results_Model_Comparison_20250509_155805
2025-05-09 15:58:05,788 - INFO - Loading and preprocessing dataset...
2025-05-09 15:58:08,169 - INFO - Raw dataset shape: (515285, 37)
2025-05-09 15:58:08,177 - INFO - 
First 5 rows before preprocessing:
  station_id  longitude  latitude                 time  AtmosphericPressure  \
0         M1      -11.2   53.1266  2001-02-07 22:00:00                999.4   
1         M1      -11.2   53.1266  2001-02-07 23:00:00               1000.0   
2         M1      -11.2   53.1266  2001-02-08 00:00:00               1000.6   
3         M1      -11.2   53.1266  2001-02-08 01:00:00               1001.2   
4         M1      -11.2   53.1266  2001-02-08 02:00:00               1001.4   

   WindDirection  WindSpeed  Gust  Wavelength  WavePeriod  ...  \
0           30.0      16.93  23.3         1.4         7.0  ...   
1           40.0      15.95  25.3         1.4         7.0  ...  

Epoch 1/200

Epoch 1: val_loss improved from inf to 0.00561, saving model to Results_Model_Comparison_20250509_155805\best_lstm_sea_temperature_model.h5


2025-05-09 16:10:24,017 - WARNING - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


9662/9662 - 734s - 76ms/step - loss: 0.0043 - mae: 0.0506 - val_loss: 0.0056 - val_mae: 0.0616 - learning_rate: 1.0000e-03
Epoch 2/200

Epoch 2: val_loss improved from 0.00561 to 0.00374, saving model to Results_Model_Comparison_20250509_155805\best_lstm_sea_temperature_model.h5


2025-05-09 16:21:53,849 - WARNING - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


...

2025-05-09 19:25:10,449 - WARNING - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


9662/9662 - 940s - 97ms/step - loss: 0.0032 - mae: 0.0442 - val_loss: 0.0034 - val_mae: 0.0473 - learning_rate: 1.0000e-03
Epoch 15/200

Epoch 15: val_loss improved from 0.00344 to 0.00334, saving model to Results_Model_Comparison_20250509_155805\best_lstm_sea_temperature_model.h5


2025-05-09 19:40:57,402 - WARNING - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


9662/9662 - 947s - 98ms/step - loss: 0.0032 - mae: 0.0437 - val_loss: 0.0033 - val_mae: 0.0471 - learning_rate: 5.0000e-04
Epoch 16/200

Epoch 16: val_loss did not improve from 0.00334
9662/9662 - 911s - 94ms/step - loss: 0.0032 - mae: 0.0437 - val_loss: 0.0035 - val_mae: 0.0482 - learning_rate: 5.0000e-04
Epoch 17/200

Epoch 17: val_loss did not improve from 0.00334
9662/9662 - 792s - 82ms/step - loss: 0.0032 - mae: 0.0436 - val_loss: 0.0037 - val_mae: 0.0488 - learning_rate: 5.0000e-04
Epoch 18/200

Epoch 18: val_loss did not improve from 0.00334
9662/9662 - 985s - 102ms/step - loss: 0.0032 - mae: 0.0436 - val_loss: 0.0035 - val_mae: 0.0478 - learning_rate: 5.0000e-04
Epoch 19/200

Epoch 19: val_loss did not improve from 0.00334
9662/9662 - 979s - 101ms/step - loss: 0.0032 - mae: 0.0436 - val_loss: 0.0036 - val_mae: 0.0486 - learning_rate: 5.0000e-04
Epoch 20/200

Epoch 20: val_loss did not improve from 0.00334
9662/9662 - 988s - 102ms/step - loss: 0.0032 - mae: 0.0436 - val_loss: 0.

2025-05-10 01:29:10,992 - INFO - Starting RandomForestRegressor training...
2025-05-10 01:29:56,041 - INFO - Starting XGBRegressor training...
2025-05-10 01:30:02,248 - INFO - Evaluating LSTM model...
2025-05-10 01:32:14,230 - INFO - 
LSTM Test Set Regression Metrics:
2025-05-10 01:32:14,231 - INFO - R² Score: 0.8917
2025-05-10 01:32:14,233 - INFO - Mean Absolute Error: 0.6118
2025-05-10 01:32:14,234 - INFO - Mean Absolute Percentage Error: 5.1706%
2025-05-10 01:32:14,235 - INFO - Root Mean Squared Error: 0.7920
2025-05-10 01:32:14,235 - INFO - Explained Variance Score: 0.8918
2025-05-10 01:32:17,564 - INFO - LSTM visualizations saved successfully.
2025-05-10 01:32:17,565 - INFO - Evaluating RandomForest model...
2025-05-10 01:32:17,826 - INFO - 
RandomForest Test Set Regression Metrics:
2025-05-10 01:32:17,827 - INFO - R² Score: 0.9720
2025-05-10 01:32:17,829 - INFO - Mean Absolute Error: 0.2890
2025-05-10 01:32:17,829 - INFO - Mean Absolute Percentage Error: 2.4422%
2025-05-10 01:32: